In [ ]:
%pip install --upgrade pip --quiet
%pip install pyspark --quiet
%pip install -U -q PyDrive --quiet
%pip install numpy pandas --quiet

In [ ]:
SCALA_VERSION = "2.12"
KAFKA_VERSION = "3.7.0"
PYSPARK_SCALA_VERSION = "2.12"
SPARK_VERSION = "3.5.1"

%env SCALA_VERSION=$SCALA_VERSION
%env KAFKA_VERSION=$KAFKA_VERSION
%env PYSPARK_SCALA_VERSION=$PYSPARK_SCALA_VERSION
%env SPARK_VERSION=$SPARK_VERSION

In [ ]:
!./start-kafka.sh
!tail -n 100 kafka/logs/server.log | grep -i "Kafka Server started"

In [ ]:
TOPIC1 = "topic1"
TOPIC2 = "topic2"
BOOTSTRAP_SERVER = "127.0.0.1:9092"

In [ ]:
!./create-topics.sh {BOOTSTRAP_SERVER} {TOPIC1} {TOPIC2}

In [ ]:
%env TOPIC1=$TOPIC1
%env TOPIC2=$TOPIC2
%env BOOTSTRAP_SERVER=$BOOTSTRAP_SERVER
!rm -rf generator.log

In [ ]:
%%script bash --bg --proc data_producer_script --out data_producer_output --err data_producer_error
echo "Starting data producer"
bash populate-topics.sh --topic1 $TOPIC1 --topic2 $TOPIC2 --bootstrap_server $BOOTSTRAP_SERVER
sleep 1

In [ ]:
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext

packages = [
    f'org.apache.spark:spark-sql-kafka-0-10_{PYSPARK_SCALA_VERSION}:{SPARK_VERSION}',
    f'org.apache.kafka:kafka-clients:{KAFKA_VERSION}'
]
# scc.stop(); spark.stop()
spark = SparkSession.builder \
    .master("local[*]") \
    .config("spark.jars.packages", ",".join(packages)) \
    .appName("PDD task-2") \
    .getOrCreate()

scc = StreamingContext(spark.sparkContext, 2)

# import json
#
# print(json.dumps({kv[0]: kv[1] for kv in spark.sparkContext.getConf().getAll()}, sort_keys=True, indent=4))

In [ ]:
!bash kafka/bin/kafka-console-consumer.sh --topic {TOPIC1} --max-messages 10 --bootstrap-server {BOOTSTRAP_SERVER}

In [ ]:
import pyspark.sql.functions as F

df = spark.readStream.format("kafka") \
  .option("kafka.bootstrap.servers", "127.0.0.1:9092") \
  .option("subscribe", "topic1") \
  .load()

In [ ]:
x = (
    df.select(F.col("*"))
    .writeStream
    .outputMode("update")
    .format("console")
    .foreachBatch(lambda df, b: df.show())
    .start()
)

x.awaitTermination(10)
x.stop()

In [2]:
# data_producer_script.kill()
# scc.stop()
# spark.stop()